In [1]:
abstract type Instr end

In [2]:
function run(instr::Instr, s::Tuple)::Tuple end

run (generic function with 1 method)

In [3]:
struct Seq{I1 <: Instr, I2 <: Instr} <: Instr end

In [4]:
Seq(::I1, ::I2) where {I1 <: Instr, I2 <: Instr} = Seq{I1, I2}()

Seq

In [5]:
run(::Seq{I1, I2}, s::Tuple) where {I1 <: Instr, I2 <: Instr} =
    run(I2(), run(I1(), s))

run (generic function with 2 methods)

In [6]:
struct Push{x} <: Instr end

In [7]:
Push(x) = Push{x}()

Push

In [8]:
run(::Push{x}, s::Tuple) where {x} = (x, s)

run (generic function with 3 methods)

In [9]:
run(Push(20), (10, ()))

(20, (10, ()))

In [10]:
struct Op2{f} <: Instr end

In [11]:
Op2(f) = Op2{f}()

Op2

In [12]:
function run(::Op2{f}, s::Tuple) where {f}
    x2 = s[1]
    x1 = s[2][1]
    (f(x1, x2), s[2][2])
end

run (generic function with 4 methods)

In [13]:
run(Seq(Push(100), Seq(Push(200), Op2((+)))), ())

(300, ())

In [14]:
run(Seq(Push(:Aa), Seq(Push(:Bb), Op2((==)))), ())

(false, ())

In [15]:
run(Seq(Push(200), Op2((+))), (100, ()))

(300, ())

**The 1st Futamura projection has been achieved!**

In [16]:
@code_typed run(Seq(Push(100), Seq(Push(200), Op2((+)))), ())

CodeInfo(
1 ─     return (300, ())
) => Tuple{Int64, Tuple{}}

In [17]:
@code_typed run(Seq(Push(:Aa), Seq(Push(:Bb), Op2((==)))), ())

CodeInfo(
1 ─     return (false, ())
) => Tuple{Bool, Tuple{}}

In [19]:
@code_typed run(Seq(Push(200), Op2((+))), (100, ()))

CodeInfo(
1 ─ %1 = Base.getfield(s, 1, true)::Int64
│   %2 = Base.add_int(%1, 200)::Int64
│   %3 = Core.tuple(%2, ())::Tuple{Int64, Tuple{}}
└──      return %3
) => Tuple{Int64, Tuple{}}

In [18]:
@code_typed run(Op2((+)), (200, (100, ())))

CodeInfo(
1 ─ %1 = Base.getfield(s, 1, true)::Int64
│   %2 = Base.getfield(s, 2, true)::Tuple{Int64, Tuple{}}
│   %3 = Base.getfield(%2, 1, true)::Int64
│   %4 = Base.add_int(%3, %1)::Int64
│   %5 = Core.tuple(%4, ())::Tuple{Int64, Tuple{}}
└──      return %5
) => Tuple{Int64, Tuple{}}